In [1]:
import pandas as pd
import re
import numpy as np
import cPickle

In [2]:
df = pd.read_csv('data/mbu.csv', usecols=['Rating','Review'], sep=',')

In [3]:
df=df.dropna()

In [4]:
x=df.values

In [5]:
s=x[:,1]

In [6]:
sent=[]
vocab = {}

In [7]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),\.!?]", " ", string)
    #string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"e\.g\.,", " ", string)
    string = re.sub(r"a\.k\.a\.", " ", string)
    string = re.sub(r"i\.e\.,", " ", string)
    string = re.sub(r"i\.e\.", " ", string)
    #string = re.sub(r"\'ve", " \'ve", string)
    #string = re.sub(r"\'", "", string)
    #string = re.sub(r"\'re", " \'re", string)
    #string = re.sub(r"\'d", " \'d", string)
    #string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", "", string)
    #string = re.sub(r",", " , ", string)
    string = re.sub(r"br", "", string)
    string = re.sub(r"!", "", string)
    #string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", "", string)
    #string = re.sub(r"\(", " ( ", string)
    string = re.sub(r"\)", "", string)
    #string = re.sub(r"\)", " ) ", string)
    string = re.sub(r"\?", "", string)
    #string = re.sub(r"\?", " ? ", string)
    string = re.sub(r"\.", "", string)
    #string = re.sub(r"\.", " . ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"u\.s\.", " us ", string)
    return string.strip().lower()

In [8]:
for i in s:
    temp = clean_str(i).split()
    sent.append(temp)
    t = set(temp)
    for word in t:
        if word in vocab:
            vocab[word] += 1
        else:
            vocab[word] = 1

In [9]:
ixtoword = {}
ixtoword[0] = 'END'
ixtoword[1] = 'UNK'
wordtoix = {}
wordtoix['END'] = 0
wordtoix['UNK'] = 1
ix = 2

In [10]:
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [11]:
def convert_word_to_ix(data):
    result = []
    for sent in data:
        temp = []
        for w in sent:
            if w in wordtoix:
                temp.append(wordtoix[w])
            else:
                temp.append(1)
        temp.append(0)
        result.append(temp)
    return result

In [12]:
train_sent = convert_word_to_ix(sent)

In [13]:
train_y=x[:,0]

In [14]:
train_l=[int(round(y)) for y in train_y]

In [15]:
train_l[:10]

[5, 4, 5, 3, 4, 1, 5, 5, 5, 5]

In [16]:
train_lab=np.zeros((len(train_l),5))

In [17]:
for i,j in zip(range(len(train_lab)),train_l):
    train_lab[i][j-1]=1 

In [18]:
train_lab[:10]

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [19]:
l=len(train_lab)

In [20]:
s1=int(round(len(train_lab)*0.8))

In [21]:
s1

155022

In [22]:
s2=int(round(len(train_lab)*0.9))

In [23]:
s2

174399

In [24]:
train_x = train_sent[:s1]
train_y = train_lab[:s1]
val_x = train_sent[s1:s2]
val_y = train_lab[s1:s2]
test_x = train_sent[s2:]
test_y = train_lab[s2:]

In [25]:
cPickle.dump([train_x, val_x, test_x, train_y, val_y, test_y, wordtoix, ixtoword], open("data/mbu.p", "wb"))